## Example for PH$_3$ grom G2 test set optimization (in a box) using Conquest

In [ ]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt

from distutils.spawn import find_executable
from numpy import amax, amin, matmul, exp, pi, array, zeros, sqrt

from ase.calculators.conquest import Conquest
from ase.units import Rydberg
from ase.build import bulk
from ase.dft.bandgap import bandgap
from ase.io import read

from ase import Atoms
from ase.spacegroup import crystal

from ase.visualize import view
from ase.collections import g2

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import print_occ, get_gapwind
from cq_ase_external_lib import cq_postproc_xsf, cq_print_output_data
from cq_ase_external_lib import read_conquest, conquest_orthorhombic_check

In [ ]:
%%bash 
ase --version

#### Check if visualisation tools are installed

In [ ]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

#### Define Conquest environment

In [ ]:
CQ_ROOT = '/Users/lioneltruflandier/CONQUEST-develop'
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 '+CQ_ROOT+'/src/Conquest'
os.environ['CQ_PP_PATH'] = CQ_ROOT+'/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = CQ_ROOT+'/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_g2_optpos'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

####  Load ASE `Atoms` object from G2 test set

- Load PH$_3$ structure from G2

In [ ]:
#print(g2.names[:])

struct = g2['PH3']

print('n atoms = {} ?'.format(len(struct)))

- Define a box with molecule inside with 5 Ang. of vacuum both sides

In [ ]:
struct.center(vacuum=5.)

- Check the system with `view`

In [ ]:
view(struct)
view(struct_last,viewer='x3d')

- Extract and print main structural data from input structure `struct`

In [ ]:
print_struct_data(struct,verbose=1)

- Save input `struct` in CIF and XSF format 

In [ ]:
struct.write(working_directory+'/input.xsf')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input structure (if possible)

In [ ]:
if ( 'VESTA' in cmd_app ):
    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

#### Setup Conquest atomic basis set

In [ ]:
basis = {
         'P' : { 'gen_basis'            : True,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },
          }

#### Setup calculation using Conquest as calculator

In [ ]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  1e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       })    

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

#### Launch structure optimization

In [ ]:
dft_energy = struct.get_potential_energy()

#### Extract last structure from Conquest file `coord_next.dat` 
> you need to provide `atomic_order` as in the Conquest input file

In [ ]:
last_struct='coord_next.dat'
last_struct=working_directory+'/'+last_struct

if( os.path.isfile(last_struct) ):

    struct_last = read_conquest(last_struct, fractional=True, atomic_order=['H','P'])
    
    struct_last.write(working_directory+'/output_.cif')
    struct_last.write(working_directory+'/output_.xsf')
    
else:
    print('WARNING: {} not found !'.format(traj_file))

view(struct_last,viewer='x3d')
view(struct_last)

- Check the molecule with `view`

In [ ]:
view(struct_last)
view(struct_last,viewer='x3d')

- run `xcrysden` to visualize the optimization movie (if possible)

In [ ]:
if ( cmd_vis['xcrysden'] and os.path.isfile(traj_file) ):
    subprocess.run(['xcrysden', '--axsf', traj_file])

#### Read Conquest output results and print the data

In [ ]:
struct.calc.read_results(struct)

cq_print_output_data(struct,2)

#### Extract and print main structural data from optimized structure `last_struct`

In [ ]:
print_struct_data(struct_last,verbose=1)

#### Extract Fermi energy, band gap and energy levels

In [ ]:
# Fermi energy as evaluated in Conquest code 'Fermi energy for spin'
fermi_energy = calc.get_fermi_level()
eigenvalues  = calc.get_eigenvalues()

print('Total electronic energy = {:14.6f} eV'.format(dft_energy))
print('Fermi energy            = {:14.6f} eV'.format(fermi_energy))
print_occ(eigenvalues,fermi_energy,0.01,method='aufbau',label='G')

# Evaluate band gap
gap, p1, p2  = bandgap(struct.calc,efermi=fermi_energy,direct=True)

## 